Importing required Libraries

In [ ]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import ParameterGrid, StratifiedKFold
from sklearn.metrics import mean_squared_log_error, make_scorer
import lightgbm as lgb
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV


Reading the data set and creating summy variables

In [ ]:
np.random.seed(1)
path = '../input/masterfinal/'
df = pd.read_csv(path+ 'master_final.csv')
final_test = df.copy()
df.sort_values(by = ['air_store_id','calendar_date'], inplace = True)

In [ ]:
final_test['air_store_idcheck'] = final_test['air_store_id']
df1 = pd.get_dummies(df, columns = ['air_genre_name', 'air_area_name','month','year', 'dayofweek', 'week_of_month', 'air_store_id'])
final_test_upd = pd.get_dummies(check1, columns = ['air_genre_name', 'air_area_name','month','year', 'dayofweek', 'week_of_month', 'air_store_id'])

Creating train, test and validation sets

In [ ]:
df2 = df1[pd.to_datetime(df1['calendar_date']) < pd.to_datetime('2017-04-23')] 
final_check = check[pd.to_datetime(check['calendar_date']) >= pd.to_datetime('2017-04-23')]
del(df2['day_of_week'])
del(df2['hpg_store_id'])
del(df2['earlist_open_date'])
df2.fillna(0, inplace = True)
test = df2[pd.to_datetime(df2['calendar_date']) >= pd.to_datetime('2017-03-15')]
train = df2[pd.to_datetime(df2['calendar_date']) < pd.to_datetime('2017-03-15')]
del(train['calendar_date'])
del(test['calendar_date'])

In [ ]:
test = test[test['closed_flag'] != 2]
x_train = train.copy()
x_test = test.copy()
del(x_train['visitors'])
del(x_test['visitors'])
y_train = train['visitors']
y_test = test['visitors']
colnames_train = x_train.columns 
colnames_test = x_test.columns


In [ ]:
new_colnames = {}
i = 1
for col in x_train.columns:
    new_colnames[col] = 'feature'+ str(i)
    i +=1
    

In [ ]:
x_train.rename(columns = new_colnames, inplace = True)
x_test.rename(columns = new_colnames, inplace = True)

In [ ]:
d_train = lgb.Dataset(x_train, label= y_train)
d_valid = lgb.Dataset(x_test, label = y_test)

Setting parameters

In [ ]:
params = {'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': ['rmsle','mean_squared_error'],
    'max_depth': 6, 
    'learning_rate': 0.1,
    'verbose': 0, 
    'num_leaves' : 20,
    'early_stopping_round': 30}
n_estimators = 100

Setting parameters for grid search

In [ ]:
gridParams = {
    'learning_rate': [ 0.1,0.01,0.001,0.00001],
    'num_leaves': [10,20,30,40,50],
    'max_depth' :[5,10,15,20]
}
result = {}
result_list = []


Iterating the model over different parameters and finding the best model

In [ ]:
num_iter = 0
for lr in gridParams['learning_rate']:
    for num in gridParams['num_leaves']:
        for dep in gridParams['max_depth']:
            params['learning_rate'] = lr
            params['num_leaves'] = num
            params['max_depth'] = dep
            model = lgb.train(params, d_train, n_estimators, valid_sets = [d_valid])
            y_pred = model.predict(x_test, num_iteration = model.best_iteration)
            for i in range(len(y_pred)):
                if y_pred[i] < 0:
                    y_pred[i]= 0
            key_dic = 'LR: ' + str(lr) + ', nleaf: ' + str(num) + ', max_dep: ' + str(dep)
            rmsl = (mean_squared_log_error(y_true = y_test,y_pred= y_pred))**0.5
            result[key_dic] = rmsl
            result_list.append(rmsl)
            if num_iter ==0:
                model_best = model 
                y_best_pred = y_pred
            elif result_list[num_iter] == min(result_list):
                model_best = model
                y_best_pred = y_pred
            num_iter += 1
            print(num_iter, min(result_list))
            

Getting the characteristics of the best model and fine tuning the model again

In [ ]:
min(result, key = result.get)

In [ ]:
min(result.values())

In [ ]:
gridParams2 = {'learning_rate': [0.1, 0.01, 1],
 'num_leaves': [50,60,75,100],
 'max_depth': [12,15,17]}
result2 = {}
result_list2 = []

In [ ]:
num_iter2 = 0
for lr in gridParams2['learning_rate']:
    for num in gridParams2['num_leaves']:
        for dep in gridParams2['max_depth']:
            params['learning_rate'] = lr
            params['num_leaves'] = num
            params['max_depth'] = dep
            model = lgb.train(params, d_train, n_estimators, valid_sets = [d_valid])
            y_pred = model.predict(x_test, num_iteration = model.best_iteration)
            for i in range(len(y_pred)):
                if y_pred[i] < 0:
                    y_pred[i]= 0
            key_dic = 'LR: ' + str(lr) + ', nleaf: ' + str(num) + ', max_dep: ' + str(dep)
            rmsl = (mean_squared_log_error(y_true = y_test,y_pred= y_pred))**0.5
            result2[key_dic] = rmsl
            result_list2.append(rmsl)
            if num_iter2 ==0:
                model_best2 = model 
                y_best_pred2= y_pred
            elif result_list2[num_iter2] == min(result_list2):
                model_best2 = model
                y_best_pred2 = y_pred
            num_iter2 += 1
            print(num_iter2)
            

Finidng the best model and reviewing the result

In [ ]:
min(result2.values())

In [ ]:
min(result2, key = result2.get)

Storing results of both iterations

In [ ]:
result_pd1 = pd.DataFrame(result.values(), index = result.keys()).reset_index().rename(columns= {'index': 'HyperParameter',0: 'RMSLE'})
result_pd1['iteration'] =1 
result_pd2 = pd.DataFrame(result.values(), index = result.keys()).reset_index().rename(columns= {'index': 'HyperParameter',0: 'RMSLE'})
result_pd2['iteration'] =2
results_lgb = pd.concat([result_pd1,result_pd2])

Getting the required metrics of the best model

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error
print('R squared : ',r2_score(y_true = y_test, y_pred = y_best_pred2))
print('RMSLE: ',mean_squared_log_error(y_test, y_best_pred2)**0.5)
print('MSE: ',mean_squared_error(y_test, y_best_pred2))



'LR: 0.1, nleaf: 50, max_dep: 15'
0.53577

Saving the model for later use

In [ ]:
 model_best2.save_model('model_best2.txt')

In [ ]:
bst = lgb.Booster(model_file='../input/modelfile/model_best2 (1).txt')

Getting the best features

In [ ]:
len(bst.feature_importance())
lgb_feature_imp = pd.DataFrame({'colname':colnames_train,'feat_imp':bst.feature_importance()})
lgb_feature_imp.sort_values(by = ['feat_imp'], ascending = False, inplace = True)
lgb_feature_imp.head()

Preparing the data set for final submission

In [ ]:
final_test_upd2 = final_test_upd.copy()
final_test_upd2.drop(columns = ['calendar_date','day_of_week','hpg_store_id','air_store_idcheck','visitors','earlist_open_date'], inplace = True)
final_test_upd2['closed_flag'] = 0
final_test_upd2.rename(columns = new_colnames, inplace = True)

Final Prediction

In [ ]:
y_final_pred = bst.predict(final_test_upd2)
for i in range(len(y_final_pred)):
    if y_final_pred[i]<0:
        y_final_pred[i] = 0 


In [ ]:
final_test['final_index'] = final_test.apply(lambda x: str(x['air_store_idcheck']) + '_' + str(x['calendar_date']), axis = 1)
sub_file = pd.merge(final_test[['final_index']],  pd.DataFrame(y_final_pred,index = final_check.index ), how = 'outer',left_index = True, right_index = True)
sub_file.shape

In [ ]:
sub_file.rename(columns = {'final_index': 'id',0:'visitors_pred'}, inplace = True)

Loading the sample file from Kaggle, appending the predictions and saving the data set

In [ ]:
sample_sub = pd.read_csv('../input/sample/sample_submission.csv')
sub_file2 = sample_sub.merge(sub_file, how = 'inner', on = 'id')
sub_file2= sub_file2.drop(columns = 'visitors').rename(columns = {'visitors_pred': 'visitors'})
sub_file2.to_csv('Submission_File_Fazel_closed.csv', index = False)
